# **表情识别项目**
### 1、背景介绍
#### 1.1 任务目标：基于嘴唇的表情识别  
<img src="/mnt/mydisk2/myPytorch/表情分类/pic/嘴唇.png" alt="FAO" width="790">  

#### 应用场景：社交娱乐，安全控制，司法系统等

#### 1.2 项目流程
<img src="/mnt/mydisk2/myPytorch/表情分类/pic/项目流程.png" alt="FAO" width="790">   

### 2、数据处理与读取
#### 2.1.1 数据获取的常见方法
- 数据集（ImageNet等）  
&emsp; - 数据质量高  
&emsp; - 成本低  
- 外包平台(阿里众包等)  
&emsp; - 大规模  
&emsp; - 成本高  
- 自己采集或爬虫  
&emsp; - 成本低  
&emsp; - 速度快
#### 2.1.2 人脸表情识别开源数据集
- 表情数据集：KDEF, RaFD, RAF, EMotioNet等
- 人脸数据集：Celeba等  
<img src="/mnt/mydisk2/myPytorch/表情分类/pic/人脸识别.png" alt="FAO" width="790">   

#### 2.1.3 爬虫获取
#### 爬虫工具：<u>https://github.com/QianyanTech/Image-Downloader</u>
- 图片种类丰富(所有类型图片)，来源于搜索引擎
- 爬取速度快(5min/1000张)
- 接口稳定，使用简单
- 数量限制  

<img src="/mnt/mydisk2/myPytorch/表情分类/pic/爬虫工具.png" alt="FAO" width="500">   

#### 2.2.1 数据预处理-归一化
- 只保留有效的数据
- 去除损坏图片，防止读取失败
- 类型归一化(方便遍历，*.jpg | *.png，统一压缩方式)
- 去除尺寸异常图片(如长宽比大于10)
- 命名归一化(方便归类)

#### 2.2.2 数据预处理-人脸检测
- OpenCV人脸检测，Dlib关键点检测  
<img src="/mnt/mydisk2/myPytorch/表情分类/pic/Dlib人脸关键点.png" alt="FAO" width="400">   


In [1]:
import cv2
import dlib
import numpy as np
# 人脸检测
cascade_path = '/mnt/mydisk2/myPytorch/表情分类/haarcascade_frontalface_default.xml'
#创建一个人脸检测器
cascade = cv2.CascadeClassifier(cascade_path)  

# 关键点检测，该模型共68个关键点
PREDICTOR_PATH = '/mnt/mydisk2/myPytorch/表情分类/shape_predictor_68_face_landmarks.dat'
#创建一个关键点检测器
predictor = dlib.shape_predictor(PREDICTOR_PATH) 

def get_landmark(im): #输入图像
    # 对输入图像进行人脸检测，并返回人脸矩形框的位置信息
    rects = cascade.detectMultiScale(im,1.3,5)   
    x,y,w,h =rects[0]
    # 用 dlib.rectangle 函数创建一个矩形对象rect
    rect = dlib.rectangle(int(x),int(y),int(x+w),int(y+h))
    # 获取人脸区域中的关键点坐标，并将其转换为一个 (68, 2) 的矩阵格式，其中每一行表示一个关键点的横纵坐标
    return np.matrix([[p.x,p.y] for p in predictor(im,rect).parts()])

&emsp;&emsp;haarcascade_frontalface_default.xml 是 OpenCV（Open Source Computer Vision Library）中的一个级联分类器（cascade classifier）文件，用于检测图像或视频中的人脸。

&emsp;&emsp;级联分类器是一种基于机器学习的对象检测方法，它使用 Haar 特征来识别对象。haarcascade_frontalface_default.xml 文件是 OpenCV 中预训练好的一个级联分类器，可以用于检测正面人脸。  

&emsp;&emsp;predictor(im, rect) 返回的是一个 full_object_detection 对象，这个对象包含了人脸矩形内部的所有关键点坐标。而 parts() 是 full_object_detection 对象的一个方法，用于获取所有关键点的坐标。这个方法返回一个包含所有关键点的列表，每个关键点以 dlib.point 对象的形式表示，其中包含了 x 和 y 两个坐标值。

#### 2.2.3 数据预处理-人脸检测
- 提取出嘴唇区域框，并适当扩大区域  

<img src="/mnt/mydisk2/myPytorch/表情分类/pic/嘴唇提取.png" alt="FAO" width="790">

In [2]:
'''
for i in range(48,67):
    x = landmarks[i,0]
    y = landmarks[i,1]
'''

'\nfor i in range(48,67):\n    x = landmarks[i,0]\n    y = landmarks[i,1]\n'

#### 2.3 完整的数据集大小
- 15000多张图，9:1均匀分为训练集与测试集  
- 无表情Onone：4763 训练集：4287 测试集：476  
- 嘟嘴1pouting：3154 训练集：2839 测试集：315  
- 微笑2smile：4821 训练集：4357 测试集：484  
- 大笑3openmouth：2348 训练集：2114 测试集：234  
- 格式：统一为128*128(不是必要操作)，jpg图像  
代码见：split_train_val.py
#### 2.3.1 数据读取
- 图像分类任务，通过torchvision包来读书数据  

<img src="/mnt/mydisk2/myPytorch/表情分类/pic/torchvision.png" alt="FAO" width="300">   

- Train和Val文件夹都包含4个类的子文件夹，自动被torchvision转成标签

In [ ]:
import torchvision
import os
import torch

data_dir = '/mnt/mydisk2/myPytorch/表情分类/data' 
# 使用torchvision的dataset_ImageFolder接口读取数据
image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x),
                    data_transforms[x]) for x in ['train','val']}

&emsp;&emsp; ImageFolder 是一个用于从磁盘读取图像文件的数据集类，可以将文件夹中的图像自动按照类别进行分组。这里使用 ImageFolder 类来加载数据集，它假设每个子目录的名称（0，1，2，3）即为其所属类别的名称，这样就能够自动地为数据集中的每个样本分配标签，而不需要手动标记每个样本的标签。该数据对象集返回一个元组(image, label)。  
&emsp;&emsp; 其中，os.path.join(data_dir, x) 将 data_dir 和 'train' 或 'val' 相结合，创建出对应的文件夹路径。data_transforms[x] 则是一个由数据增强操作组成的 PyTorch 的 transforms.Compose 对象，该对象将被应用于数据集中的每个图像。这些操作可以包括对图像进行缩放、裁剪、旋转、翻转、归一化等处理，以提高模型的鲁棒性和泛化能力。  
&emsp;&emsp;image_datasets 是一个字典，包含了两个键值对。键 'train' 对应的值是一个训练集数据集对象，键 'val' 对应的值是一个验证集数据集对象。这些数据集对象包含了对应数据集中的图像数据和标签信息等。data_transforms是必要的数据增强函数。

#### 2.3.2 数据增强
- 添加随机缩放裁剪，随机翻转，去均值方差归一化操作

In [4]:
from torchvision import transforms
# 创建数据预处理函数，训练预处理包括随机裁剪缩放，随机翻转，归一化，验证预处理包括中心裁剪，归一化
data_transforms = {
    'train':transforms.Compose([
            transforms.RandomResizedCrop(48),   # 随机裁剪，将图像随机裁剪为指定大小（48 x 48）的正方形。
            transforms.RandomHorizontalFlip(),  # 随机水平翻转，以一定概率（默认为0.5）随机翻转图像。
            transforms.ToTensor(),              # 将 PIL 图像转换为 PyTorch 的 Tensor 格式，将图像数据归一化到 [0, 1] 的范围内。
            transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])  # 对归一化后的图像数据进行标准化，使其均值为 0.5，标准差为 0.5。
    ]),
    'val':transforms.Compose([   # 测试集的数据预处理都不能用随机
            transforms.Resize(48),  
            transforms.CenterCrop(48),   # 中心裁剪
            transforms.ToTensor(),
            transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
}

"\ndata_transforms = {\n    'train':tranforms.Compose([\n        transforms.RandomResizedCrop(48),\n        transforms.RandomHoizontalFlip(),\n        transforms.ToTensor(),\n        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])                                         \n    ]),\n    'val':tranforms.Compose([\n        transforms.Scale(64),\n        transforms.CenterCrop(48),\n        transforms.ToTensor(),\n        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])                                         \n    ])\n}\n"

#### 2.3.3 数据封装
&emsp;&emsp;使用data.DataLoader创建两个数据加载器对象，一个用于训练集，另一个用于验证集。数据加载器是一个可以迭代访问数据集中的批次数据的对象，它可以将数据集划分成小批次，使得训练过程可以更高效地进行。

In [3]:
# 创建数据指针，设置batch大小，shuffle，多进程数量
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                batch_size=64,
                                                shuffle=True,
                                                num_workers=4) for x in ['train','val']}

"\ndata_dir = './data'\n# 分别读取'./data/train'和'./data/val'文件夹\nimage_datasets = {x:dataset.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}\n# 设置batchsize大小为4,数据读取线程数为4,使用随机打乱操作\ndataloaders = {x:torch.utils.data.DataLoader(image_datasets[x],\n                batch_size=16,\n                shuffle=True,\n                run_workers=4) for x in ['train','val']} \n"

&emsp;&emsp;batch_size 参数指定了每次迭代加载的图像数量，shuffle 参数指定是否在每次迭代前随机打乱数据集，run_workers 参数指定了加载数据时使用的线程数。  
&emsp;&emsp;数据加载器对象将返回一个元组 (inputs, labels)，其中 inputs 是一个大小为 (batch_size, channels, height, width) 的张量，labels 是一个大小为 (batch_size,) 的张量，分别包含了当前批次数据中的输入和标签。channels 表示图像中的通道数，通常为 1 或 3，height 和 width 表示图像的高度和宽度，具体取决于输入图像的大小和预处理方式。

### 3. 模型搭建与训练
#### 3.1 网络搭建
-  定义一个简单的模型，由3个卷积层，3个BN层，3个全连接层构成
<img src="/mnt/mydisk2/myPytorch/表情分类/pic/神经网络定义.jpg" alt="FAO" width="600">   

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class simpleconv3(nn.Module):
    # 初始化函数
    def __init__(self,nclass):
        super(simpleconv3,self).__init__()
        self.conv1 = nn.Conv2d(3,12,3,2)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12,24,3,2)  #输入图片为12*23*23， 输出特征图的大小为24*11*11，卷积核大小为3*3，步长为2
        self.bn2 = nn.BatchNorm2d(24)
        self.conv3 = nn.Conv2d(24,48,3,2)   #输入图片大小为24*11*11，输出特征图大小为48*5*5，卷积核大小为3*3，步长为2
        self.bn3 = nn.BatchNorm2d(48)
        self.fc1 = nn.Linear(48*5*5, 1200)  # 输入向量长为48*5*5，输出向量长为1200
        self.fc2 = nn.Linear(1200,128)      # 输入向量长为1200，输出向量长为128
        self.fc3 = nn.Linear(128,nclass)    # 输入向量长为128，输出向量长为nclass，等于类别数
    
    # 前向函数
    def forward(self, x):
        # relu 函数，不需要进行实例化，直接进行调用
        # conv，fc层需要调用nn.Module进行实例化
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(-1,48*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#### 3.2 模型训练
导入依赖

In [ ]:
from __future__ import print_function,division
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from net import simpleconv3
import os
import torchvision
import torch.utils.data
from torchvision import transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

使用tensorboard进行可视化:  
&emsp;&emsp;'SummaryWriter'是PyTorch内置的一个类，可以用来写入指标和数据到TensorBoard日志文件中。在这段代码中，创建了一个SummaryWriter实例，并指定了输出目录为logs(默认目录为runs),用于存储TensorBoard日志文件。  
&emsp;&emsp;在训练和评估过程中，可以使用add_scalar等方法将指标数据写入到SummaryWriter对象中，例如：  
`writer.add_scalar('loss', loss, global_step)`  
&emsp;&emsp;其中，'loss'表示指标名称，loss表示具体的指标值，global_step表示当前的全局步数，这些数据会被写入到TensorBoard日志文件中。  
&emsp;&emsp;通过TensorBoard工具可以可视化和分析指标数据，例如<U>生成损失函数曲线、显示模型结构和参数</u>等等，有助于对模型进行优化和调试。

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('logs')

### 训练主函数
分为六个步骤：  
1.数据加载：从 dataloaders[phase] 中获取数据，其中 phase 取值为 'train' 或 'val'，分别代表训练和验证阶段。

2.前向传播：将数据输入模型中进行前向传播，得到模型的预测结果。

3.反向传播：在训练阶段，通过计算模型预测结果与真实标签之间的误差，利用误差反向传播算法更新模型参数。

4.损失计算：计算模型的预测结果与真实标签之间的误差，使用 criterion 进行损失计算。

5.参数更新：在训练阶段，利用优化器 optimizer 更新模型参数。

6.计算精度与损失：计算当前 batch 的平均损失与精度。

7.可视化：将每个 epoch 的平均损失与精度写入 tensorboard 中，以便可视化训练过程。

In [ ]:
# criterion指损失函数,optimizer指优化器,schecduler学习率调度器,num_epochs指训练的轮数，默认为25
def train_model(model, criterion,optimizer,schecduler,num_epochs=25):
    for epoch in range(num_epochs):
        #在每个epoch开始时，打印出当前epoch的序号
        print('Epoch {}/{}'.format(epoch,num_epochs - 1))
        for phase in ['train','val']:
            if phase == 'train':
                schecduler.step()   # 调用学习率调度器的step()函数用于更新学习率
                model.train(True)   # 设置为训练模式,模型型会根据前向和后向传播计算的梯度来更新其权重
            else:
                model.train(False)  # 设置为验证模式,模型只会前向传播，不会反向传播更新权重
            
            running_loss = 0.0  # 损失值，每个 batch 训练完成后，将该 batch 的损失值加到 running_loss 变量中
            running_accs = 0.0  # 精度值，每个 batch 训练完成后，将该 batch 的精度值加到 running_accs 变量中
            number_batch = 0 
            
            '''
                这些变量在训练过程中非常重要，可以帮助我们实时监测模型的训练效果和进度，以便进行调整和优化。
            例如，通过计算每个 epoch 中训练集上的平均损失和平均精度，可以评估模型的训练效果是否有提升，进而
            调整学习率、损失函数等超参数，以获得更好的模型效果。同时，通过记录已经训练的批次数，可以帮助我们
            更好地控制训练过程，例如设置学习率调度器、保存模型等。 
            '''

            # 从dataloader中获取数据
            for data in dataloaders[phase]:
                inputs, labels = data      # 从数据加载器中获取输入数据和标签数据
                if use_gpu:                # 若GPU可用，则将数据移动到GPU上进行计算
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                
                optimizer.zero_grad()    # 清空梯度,以便在下一个步骤中计算新的梯度。
                outputs = model(inputs)  # 前向运行
                _,preds= torch.max(outputs.data,1)  # torch.max返回了最大值和最大值所在的位置（即概率最大的类别）
                                                    # 对模型进行softmax计算，用'_'变量接收最大值的值，用'preds'变量是用来接收类别索引的
                                                    # 参数1表示，表示在每行数据中寻找最大值，返回最大值和最大值所在的位置。
                loss = criterion(outputs,labels)  # 计算损失，将模型的输出和真实标签数据作为输入，计算差距。
                if phase == 'train':  # 在训练阶段进行误差反向传播和参数更新
                    loss.backward() # 误差反向传播，计算当前批次的所有参数的梯度
                    optimizer.step() # 参数更新，将当前批次的梯度应用于所有参数中，以更新它们的值
                
                running_loss += loss.data.item()   # 累加所有批次的损失值，以计算整个训练/验证过程的平均损失。
                running_accs += torch.sum(preds == labels).item()  # 累加所有批次的正确预测次数，以计算整个训练/验证过程的准确率。
                number_batch += 1

            # 得到每一个epoch的平均损失与精度
            epoch_loss = running_loss / number_batch
            epoch_acc = running_accs / dataset_sizes[phase]

            # 收集精度和损失用于可视化
            if phase == 'train':
                writer.add_scalar('data/trainloss',epoch_loss,epoch)  # 将当前训练批次的损失写入 TensorBoard 日志文件中，使用 data/trainloss 标签来标识这个值。
                                                                      # epoch 参数指定了当前批次的索引，这样在 TensorBoard 中可以看到每个批次的损失值变化情况。
                writer.add_scalar('data/trainloss',epoch_acc,epoch)
            else:
                writer.add_scalar('data/valloss',epoch_loss,epoch)
                writer.add_scalar('data/valloss',epoch_acc,epoch)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))

    writer.close()
    return model

#### 模型主函数：


In [ ]:
if __name__ == '__main__':
    
    image_size = 64  # 图像统一缩放大小
    crop_size = 48   # 图像裁剪大小，即训练输入大小
    nclass = 4 # 分类类别数
    model = simpleconv3(nclass) # 创建模型
    data_dir = '/mnt/mydisk2/myPytorch/表情分类/data'  # 数据目录

    # 模型缓存接口
    if not os.path.exists('models'):
        os.mkdir('models')

    # 检查GPU是否可用，如果是使用GPU，否使用CPU
    use_gpu = torch.cuda.is_available()
    if use_gpu:
        model = model.cuda()
    print(model)

    # 创建数据预处理函数，训练预处理包括随机裁剪缩放，随机翻转，归一化，验证预处理包括中心裁剪，归一化
    data_transforms = {
        'train':transforms.Compose([
            transforms.RandomResizedCrop(48),   # 随机裁剪，将图像随机裁剪为指定大小（48 x 48）的正方形。
            transforms.RandomHorizontalFlip(),  # 随机水平翻转，以一定概率（默认为0.5）随机翻转图像。
            transforms.ToTensor(),              # 将 PIL 图像转换为 PyTorch 的 Tensor 格式，将图像数据归一化到 [0, 1] 的范围内。
            transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])  # 对归一化后的图像数据进行标准化，使其均值为 0.5，标准差为 0.5。
        ]),
        'val':transforms.Compose([   # 测试集的数据预处理都不能用随机
            transforms.Resize(48),  
            transforms.CenterCrop(48),   # 中心裁剪
            transforms.ToTensor(),
            transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
        ])
    }

    # 使用torchvision的dataset_ImageFolder接口读取数据
    image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x),
                        data_transforms[x]) for x in ['train','val']}

    # 创建数据指针，设置batch大小，shuffle，多进程数量
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                batch_size=64,
                                                shuffle=True,
                                                num_workers=4) for x in ['train','val']}
    
    # 获得数据大小
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}

    # 优化目标是个交叉熵，优化方法使用带动量项的SGD，学习率迭代策略为step,每隔100个epoch,变为原来的0.1倍
    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(model.parameters(),lr=0.1,momentum=0.9)   # 定义优化器，学习率为0.1，动量为0.9
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,step_size=100,gamma=0.1)  # 每当训练过程进行100个epoch时，将优化器中的学习率降低为原来的0.1倍。
                                                                                  
                                                                                  

    model = train_model(model=model,
                        criterion=criterion,
                        optimizer=optimizer_ft,
                        schecduler=exp_lr_scheduler,
                        num_epochs=300)
    
    torch.save(model.state_dict(),'/mnt/mydisk2/myPytorch/表情分类/models/model.pt')

- #### criterion：交叉熵损失函数
`nn.CrossEntropyLoss()`:输入为模型的输出和真实标签数据
- #### optimizer_ft：优化器
&emsp;&emsp;这里使用了SGD优化器（Stochastic Gradient Descent）,用于优化神经网络的参数。  
&emsp;&emsp;_model.parameters()_ 是指定需要优化的参数，即神经网络的权重和偏置。  
&emsp;&emsp;_lr=0.1_：学习率。SGD优化器会根据学习率来更新神经网络的参数。学习率越大，神经网络的参数更新越快，但可能会导致训练不稳定；学习率越小，神经网络的参数更新越慢，但训练可能更稳定。  
&emsp;&emsp;_momentum=0.9_：动量。momentum参数表示前一次梯度更新的方向在本次更新中所占的比重。动量可以加速SGD优化器在梯度下降过程中的收敛速度。在这个例子中，动量被设置为0.9。通常，momentum参数的值介于0到1之间，一般取0.9左右的数值效果较好。
- #### 学习率调度器：  
&emsp;&emsp;学习率调度器可以根据训练过程中的指标变化（如损失函数或验证集上的准确率等）来自动地调整学习率的大小和变化方式，以使模型训练效果更好。常见的学习率调度器有 StepLR、ExponentialLR、CosineAnnealingLR 等，它们可以分别根据训练轮数、指数衰减、余弦退火等方式来动态地调整学习率的大小和变化。在实际使用中，学习率调度器通常与优化器一起使用，以实现更加高效和精确的模型训练。  
&emsp;&emsp;_optimizer_ft_ 指定了优化器，即用于更新模型参数的优化器对象。  
&emsp;&emsp;_step_size_ 指定了学习率调整的间隔，即在训练过程中每隔多少个epoch降低一次学习率。  
&emsp;&emsp;_gamma_ 指定了学习率降低的系数，即每次降低的学习率为原来的gamma倍。  
代码中每当训练过程进行100个epoch时，将优化器(optimizer_ft)中的学习率(lr)降低为原来的0.1倍(gamma=0.1)。这样可以使模型参数的更新速度逐渐变慢，从而更好地控制模型的训练过程，防止过拟合和震荡，并提高模型的泛化能力。

### 4. 模型保存

In [ ]:
torch.save(model.state_dict(),'/mnt/mydisk2/myPytorch/表情分类/models/model.pt')

_model.state_dict()_ ：该函数返回一个包含模型中所有参数和缓存项的字典。这个字典的键是参数和缓存项的名称，值是相应的张量(Tensor)。  
&emsp;&emsp;通过这行代码，我们可以将训练好的模型状态字典保存下来，以便以后可以加载它们并继续训练或者进行推理预测。同时，将模型状态字典保存下来还可以方便我们在不同的设备上加载和使用模型，从而提高了模型的复用性和可移植性。